In [31]:
# 生成数据集
import torch
import numpy as np

num_features, num_examples = 2, 1000
w_true, b_true = [2, -3.4], 4.2
features = torch.randn(num_examples, num_features, dtype=torch.float32)
labels = w_true[0] * features[:,0] + w_true[1] * features[:,1] + b_true
# 添加噪声
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)
features[0], labels[0]

(tensor([-1.1068, -0.2320]), tensor(2.7696))

In [32]:
import torch.utils.data as Data

batch_size = 10
# 特征与标签组合
datasets = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(datasets, batch_size, shuffle=True)

for x, y in data_iter:
    print(x, y)
    break

tensor([[ 0.0423,  0.0058],
        [-0.2988,  1.1971],
        [-1.8696, -0.4114],
        [-0.4634, -0.2314],
        [ 0.6169,  0.3444],
        [ 0.1787,  1.0380],
        [-0.7518, -1.3058],
        [ 1.3697, -1.3374],
        [-0.5880,  0.0984],
        [-0.5575,  0.0125]]) tensor([ 4.2692, -0.4841,  1.8642,  4.0792,  4.2638,  1.0193,  7.1576, 11.4724,
         2.7118,  3.0605])


In [33]:
# 定义模型
class LinearNet(torch.nn.Module):
    def __init__(self, num_features) -> None:
        super(LinearNet, self).__init__()
        # 定义一层网络结构
        self.linear = torch.nn.Linear(num_features, 1)
    # 定义前向传播
    def forward(self, x):
        return self.linear(x)

# 构建一个模型
net = LinearNet(num_features)
print(net)


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [34]:
# 查看所有的可学习参数
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.2844, 0.4371]], requires_grad=True)
Parameter containing:
tensor([-0.6035], requires_grad=True)


In [35]:
# 初始化模型参数
torch.nn.init.normal_(net.linear.weight, mean=0, std=0.01)
torch.nn.init.constant_(net.linear.bias, val=0)

# 定义损失函数和优化算法

loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [36]:
# 模型训练
epochs = 3
for epoch in range(1, epochs + 1):
    for x, y in data_iter:
        y_hat = net(x)
        l = loss(y_hat, y.view(y_hat.shape))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000410
epoch 2, loss: 0.000098
epoch 3, loss: 0.000135


In [37]:

print(w_true, net.linear.weight)
print(b_true, net.linear.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0002, -3.3994]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2009], requires_grad=True)
